In [5]:
from flask import Flask, render_template, request
import numpy as np
import re
import base64
from PIL import Image
from imageio import imread, imwrite
from keras.models import load_model
import json

In [6]:
def normalize(nparr):
    return np.interp(nparr, [0, 255], [-1,1])

In [7]:
app = Flask(__name__)

# mlp = load_model("./models/mlp_94.h5")
object_file = open("./object.txt", "r")
objects = object_file.readlines()
object_file.close()
N_CLASSES = len(objects)
CLASSES = {}
for idx, obj in enumerate(objects):
    CLASSES[idx] = obj.replace('\n', '')
print(CLASSES)
conv = load_model("./model/doodle.h5")
SHAPES = CLASSES

def save_numpy_array(array, file_path):
    """
    Save a NumPy array to a file.

    Parameters:
    - array: NumPy array to be saved.
    - file_path: File path to save the array to.
    """
    np.save(file_path, array)
    print(f"NumPy array saved to {file_path}")

@app.route("/", methods=["GET", "POST"])
def ready():
    if request.method == "GET":
        classes =list(SHAPES.values())
        classesText = f"Draw an object belong to one of the following categories: {', '.join(classes)}. Then see the prediction"
        return render_template("index1.html", classesText=classesText)
    if request.method == "POST":
        data = request.form["payload"].split(",")[1]
        net = request.form["net"]

        img = base64.decodebytes(data.encode('utf-8'))
        with open('static/temp.png', 'wb') as output:
            output.write(img)
        with open('temp.png', 'wb') as output:
            output.write(img)
        x = imread('temp.png', pilmode='L')
        # resize input image to 28x28
        x = np.array(Image.fromarray(x).resize((28, 28)))

        # if net == "MLP":
        #     model = mlp
        #     # invert the colors
        #     x = np.invert(x)
        #     # flatten the matrix
        #     x = x.flatten()

        if net == "CNN":
            model = conv
            x = np.expand_dims(x, axis=0)
            x = np.reshape(x, (28, 28, 1))
            # invert the colors
            x = np.invert(x)
            # brighten the image by 60%
            for i in range(len(x)):
                for j in range(len(x)):
                    if x[i][j] > 50:
                        x[i][j] = min(255, x[i][j] + x[i][j] * 0.60)

        # normalize the values between -1 and 1
        x = normalize(x)
        val = model.predict(np.array([x]))
        print(val)
        pred = SHAPES[np.argmax(val)]
        classes =list(SHAPES.values())
        print(pred)
        print(classes)
        print(list(val[0]))
        classesText = f"Draw an object belong to one of the following categories: {', '.join(classes)}. Then see the prediction"
        return render_template("index1.html", preds=list(val[0]), classes=json.dumps(classes), chart=True, putback=request.form["payload"], net=net, tempImage='./temp.png', classesText=classesText)

{0: 'triangle', 1: 'circle', 2: 'square', 3: 'apple', 4: 'banana'}


In [8]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Jan/2024 11:16:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2024 11:16:17] "GET /static/index.js HTTP/1.1" 304 -


1/1 [==============================] - 0s 81ms/step


C:\Users\Admin\AppData\Local\Temp\ipykernel_1712\970322433.py:41: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  x = imread('temp.png', pilmode='L')
127.0.0.1 - - [03/Jan/2024 11:16:26] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2024 11:16:26] "GET /static/index.js HTTP/1.1" 304 -


[[9.9550098e-01 7.2673726e-04 4.7931689e-04 4.9747305e-05 3.2432326e-03]]
triangle
['triangle', 'circle', 'square', 'apple', 'banana']
[0.995501, 0.00072673726, 0.0004793169, 4.9747305e-05, 0.0032432326]


...

127.0.0.1 - - [03/Jan/2024 11:18:41] "GET /static/temp.png HTTP/1.1" 200 -


1/1 [==============================] - 0s 24ms/step


127.0.0.1 - - [03/Jan/2024 11:18:53] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2024 11:18:53] "GET /static/index.js HTTP/1.1" 304 -


[[9.2271990e-01 1.1832883e-02 1.8867417e-03 1.7681778e-04 6.3383684e-02]]
triangle
['triangle', 'circle', 'square', 'apple', 'banana']
[0.9227199, 0.011832883, 0.0018867417, 0.00017681778, 0.06338368]


127.0.0.1 - - [03/Jan/2024 11:18:53] "GET /static/temp.png HTTP/1.1" 200 -
